In [1]:
""" Python Standard Library """
import os
import datetime as dt

import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())

from covid_model.data_imports import ExternalPopulation
from covid_model.utils import db_engine

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/brittneycontreras/PycharmProjects/covid-rmw-model


In [2]:
vacc_df = pd.read_csv("covid_model/model_dev_qc/vaccines/unprocessed_vacc_data.csv", header=[0,1,2], index_col=0)
engine = db_engine()
pop_df = ExternalPopulation(engine).fetch()

In [3]:
pop_df = pop_df.drop(columns = ['region_pop']).T.unstack().reset_index(level=1, name='age_pop').rename(columns={'level_1':'age_group'})[['age_pop','age_group']]
pop_df = pop_df.reset_index(level=0)

In [21]:
pop_df[pop_df['region_id'] is in [] ]

,region_id,age_pop,age_group
0,coe,80672.8,0-17
1,coe,221991.2,18-64
2,coe,74736.0,65+
3,con,1057832.0,0-17
4,con,3100910.0,18-64
...,...,...,...
57,wyn,23985.6,0-17
58,wyn,59268.4,18-64
59,wyn,17304.0,65+
60,wyw,38846.6,0-17


In [5]:
percent_vacc_df = pd.DataFrame()
region = []
vaccination = []
age = []
percent = []
cumul_vacc = []
i=0

for col in vacc_df:
    region.append(vacc_df.columns.values[i][0])
    vaccination.append(vacc_df.columns.values[i][1])
    age.append(vacc_df.columns.values[i][2])
    pop = pop_df[pop_df['region_id']==region[i]][pop_df['age_group']==age[i]]['age_pop'].values[0]
    total_vacc = vacc_df[col][-1]
    cumul_vacc.append(total_vacc)
    percent.append((total_vacc/pop)*100)
    i = i+1
    
percent_vacc_df['dose'] = vaccination
percent_vacc_df['age_group'] = age
percent_vacc_df['region_id'] = region
percent_vacc_df['cumul_vacc'] = cumul_vacc
percent_vacc_df['%_vacc'] = percent

/var/folders/82/f8fr93d51730xxd6dv7x7bdr0000gp/T/ipykernel_28501/932688478.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pop = pop_df[pop_df['region_id']==region[i]][pop_df['age_group']==age[i]]['age_pop'].values[0]


In [32]:
percent_vacc_df.sort_values(by=['region_id','age_group']).reset_index(drop=True).head(80)

,dose,age_group,region_id,cumul_vacc,%_vacc
0,booster1,0-17,coe,3636.500000,4.507715
1,booster3,0-17,coe,748.040000,0.927252
2,dose1,0-17,coe,36452.160000,45.185193
3,dose2,0-17,coe,20378.440000,25.260608
4,booster1,18-64,coe,63425.920000,28.571367
...,...,...,...,...,...
75,booster2,18-64,ids,5831.742574,4.947513
76,booster3,18-64,ids,10336.400000,8.769158
77,dose1,18-64,ids,81092.110000,68.796637
78,dose2,18-64,ids,70821.630000,60.083404


In [36]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(percent_vacc_df.sort_values(by=['region_id','age_group']).reset_index(drop=True).drop(columns=["cumul_vacc"]))

         dose age_group region_id   %_vacc
0    booster1      0-17       coe    4.508
1    booster3      0-17       coe    0.927
2       dose1      0-17       coe   45.185
3       dose2      0-17       coe   25.261
4    booster1     18-64       coe   28.571
5    booster2     18-64       coe    7.087
6    booster3     18-64       coe    8.790
7       dose1     18-64       coe  119.008
8       dose2     18-64       coe   78.380
9    booster1       65+       coe   69.854
10   booster2       65+       coe   49.138
11   booster3       65+       coe   40.259
12      dose1       65+       coe  144.560
13      dose2       65+       coe   87.355
14   booster1      0-17       con   15.126
15   booster3      0-17       con    6.576
16      dose1      0-17       con   40.777
17      dose2      0-17       con   38.529
18   booster1     18-64       con   44.487
19   booster2     18-64       con    8.726
20   booster3     18-64       con   18.879
21      dose1     18-64       con   76.677
22      dos

In [25]:
print(percent_vacc_df['%_vacc'].max())
bad_regions = percent_vacc_df[percent_vacc_df['%_vacc']>=99].drop(columns=['cumul_vacc']).sort_values(by=['region_id']).reset_index(drop=True)
bad_regions.head(50)

218.17463257996792


,dose,age_group,region_id,%_vacc
0,dose1,18-64,coe,119.008357
1,dose1,65+,coe,144.559730
2,dose1,0-17,cow,114.065376
3,dose1,18-64,cow,188.474110
4,dose1,65+,cow,183.944019
5,dose2,18-64,cow,115.855517
6,dose2,65+,cow,137.734219
7,dose1,65+,idn,117.108299
8,dose2,65+,idn,102.727449
9,dose1,18-64,mtn,107.443508
